### Testing custom package

In [1]:
from nav_pii_anon.spacy.spacy_model import SpacyModel

In [2]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652. "
      "Telefonnummeret mitt er 94867658 og jeg reiser "
      "hjem 24.07.2020")

In [3]:
nlp = SpacyModel()

In [4]:
graph = nlp.dependency_graph(tx)

In [6]:
nlp.top_n_nodes(tx, 4)

[('reiser', 4), ('mitt', 3), ('marius', 3), ('er', 3)]

In [20]:
sorted(list(graph.degree), key= lambda x: x[1], reverse=True)

[('reiser', 4),
 ('mitt', 3),
 ('marius', 3),
 ('er', 3),
 ('personnummer', 3),
 ('telefonnummeret', 3),
 ('94867658', 3),
 ('navn', 2),
 ('og', 2),
 ('15044216652.', 2),
 ('24.07.2020', 2),
 ('jeg', 1),
 ('hjem', 1)]

In [18]:
a = list(graph.degree).sort(key=lambda x: x[0][1])
print(a)

None


In [4]:
nlp = SpacyModel(spacy.load("nb_core_news_lg"))
reg = SpacyModel(spacy.load("nb_core_news_lg"))

In [5]:
tx = "Mitt navn er Marius og mitt personnummer er 15044216652"

In [6]:
nlp.predict(tx)

[['Marius', 'PER', 3, 4, 'NA']]


In [7]:
nlp.add_patterns()
reg.add_patterns()

In [8]:
nlp.predict(tx)

(15044216652,)
[['Marius', 'PER', 3, 4, 'NA'], ['15044216652', 'FNR', 8, 9, 'NA']]


In [9]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652."
      "Telefonnummeret mitt er 94867658 og jeg reiser"
      "hjem 19.07.2020")

In [10]:
nlp.predict(tx)

(94867658,)
(94867658, 19.07.2020)
[['Marius', 'PER', 3, 4, 'NA'], ['94867658', 'TLF', 11, 12, 'NA'], ['reiserhjem', 'ORG', 14, 15, 'NA'], ['19.07.2020', 'DTM', 15, 16, 'NA']]


In [ ]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652. "
      "Telefonnummeret mitt er 94867658 og jeg reiser "
      "hjem 24.07.2020")

In [ ]:
nlp.predict(tx)

In [ ]:
reg.disable_NER()

In [ ]:
reg.predict(tx)

In [ ]:
reg.enable_NER()

In [ ]:
reg.predict(tx)

In [ ]:
nlp.display_predictions(tx)

In [ ]:
print(nlp.replace(tx))